In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense

import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/train_germination.npy')
val_x = np.load('/home/ckwwk/data/germination_data/val_germination.npy')
test_x = np.load('/home/ckwwk/data/germination_data/test_germination.npy')

In [4]:
t1 = train_x[:,:,:,100:120]

In [5]:
t1.shape

(1000, 61, 61, 20, 1)

In [6]:
v1 = val_x[:,:,:,100:120]

In [7]:
v1.shape

(250, 61, 61, 20, 1)

In [8]:
tt1 = test_x[:,:,:,100:120]

In [9]:
tt1.shape

(286, 61, 61, 20, 1)

In [10]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/label/germination_label.csv")
label_df.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [11]:
from keras.utils import to_categorical
train_y = label_df[0:1000]
train_y = to_categorical(train_y)

In [12]:
val_y = label_df[1000:1250]
val_y = to_categorical(val_y)
test_y = label_df[1250:1536]
test_y = to_categorical(test_y)

In [13]:
t1 = t1.astype('float32')
v1 =v1.astype('float32')
tt1 = tt1.astype('float32')

t1 /= 255
v1 /= 255
tt1 /= 255

In [14]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [15]:
input_layer = Input((61, 61, 20,1))

conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu',padding ='same')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu',padding ='same')(conv_layer1)
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2),padding ='same')(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu',padding ='same')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu',padding ='same')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(1, 1, 2),padding ='same')(conv_layer4)

print(pooling_layer2._keras_shape)
conv3d_shape = pooling_layer2._keras_shape
conv_layer4 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(pooling_layer2)
conv_layer5 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer4)

flatten_layer = Flatten()(conv_layer5)

dense_layer1 = Dense(units=1024, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

W0809 17:10:16.729466 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 17:10:16.730705 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 17:10:16.734573 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 17:10:16.838293 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.com

(None, 31, 31, 5, 64)


In [16]:
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61, 61, 20, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 61, 61, 20, 8)     224       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 61, 61, 20, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 31, 31, 10, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 31, 31, 10, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 31, 31, 10, 64)    55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 31, 31, 5, 64)     0         
__________

In [17]:
#adam = Adam(lr=0.01, decay=1e-06)
#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

W0809 17:11:05.544190 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 17:11:05.553255 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [18]:
filepath = "best-model_V7_4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='/home/ckwwk/data/C2/tensor_log_V7_4',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [19]:
history = model.fit(x=t1, y=train_y, batch_size=128, epochs=1000, validation_data=(v1,val_y),callbacks=callbacks_list)

W0809 17:11:16.970623 140274464466752 deprecation.py:323] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 250 samples


W0809 17:11:19.403203 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:848: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0809 17:11:19.442473 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0809 17:11:19.445179 140274464466752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/1000
1000/1000 [==============================] - 8s 8ms/step - loss: 0.6897 - acc: 0.6220 - val_loss: 0.6855 - val_acc: 0.6160

Epoch 00001: val_acc improved from -inf to 0.61600, saving model to best-model_V7_4.hdf5
Epoch 2/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6810 - acc: 0.6320 - val_loss: 0.6778 - val_acc: 0.6160

Epoch 00002: val_acc did not improve from 0.61600
Epoch 3/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6722 - acc: 0.6320 - val_loss: 0.6706 - val_acc: 0.6160

Epoch 00003: val_acc did not improve from 0.61600
Epoch 4/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6643 - acc: 0.6320 - val_loss: 0.6668 - val_acc: 0.6160

Epoch 00004: val_acc did not improve from 0.61600
Epoch 5/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6597 - acc: 0.6320 - val_loss: 0.6660 - val_acc: 0.6160

Epoch 00005: val_acc did not improve from 0.61600
Epoch 6/1000
1000/1000 [==

1000/1000 [==============================] - 3s 3ms/step - loss: 0.6587 - acc: 0.6320 - val_loss: 0.6670 - val_acc: 0.6160

Epoch 00044: val_acc did not improve from 0.61600
Epoch 45/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6588 - acc: 0.6320 - val_loss: 0.6669 - val_acc: 0.6160

Epoch 00045: val_acc did not improve from 0.61600
Epoch 46/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6596 - acc: 0.6320 - val_loss: 0.6668 - val_acc: 0.6160

Epoch 00046: val_acc did not improve from 0.61600
Epoch 47/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6598 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00047: val_acc did not improve from 0.61600
Epoch 48/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6585 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00048: val_acc did not improve from 0.61600
Epoch 49/1000
1000/1000 [==============================] - 3s 3ms/step - l

...

1000/1000 [==============================] - 3s 3ms/step - loss: 0.6581 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00819: val_acc did not improve from 0.61600
Epoch 820/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6578 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00820: val_acc did not improve from 0.61600
Epoch 821/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6576 - acc: 0.6320 - val_loss: 0.6667 - val_acc: 0.6160

Epoch 00821: val_acc did not improve from 0.61600
Epoch 822/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6571 - acc: 0.6320 - val_loss: 0.6667 - val_acc: 0.6160

Epoch 00822: val_acc did not improve from 0.61600
Epoch 823/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6591 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00823: val_acc did not improve from 0.61600
Epoch 824/1000
1000/1000 [==============================] - 4s 4ms/ste

1000/1000 [==============================] - 4s 4ms/step - loss: 0.6581 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00862: val_acc did not improve from 0.61600
Epoch 863/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6585 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00863: val_acc did not improve from 0.61600
Epoch 864/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6579 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00864: val_acc did not improve from 0.61600
Epoch 865/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6578 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00865: val_acc did not improve from 0.61600
Epoch 866/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6579 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00866: val_acc did not improve from 0.61600
Epoch 867/1000
1000/1000 [==============================] - 3s 3ms/ste

1000/1000 [==============================] - 4s 4ms/step - loss: 0.6578 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00905: val_acc did not improve from 0.61600
Epoch 906/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6582 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00906: val_acc did not improve from 0.61600
Epoch 907/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6583 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00907: val_acc did not improve from 0.61600
Epoch 908/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6582 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00908: val_acc did not improve from 0.61600
Epoch 909/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6582 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00909: val_acc did not improve from 0.61600
Epoch 910/1000
1000/1000 [==============================] - 3s 3ms/ste

1000/1000 [==============================] - 4s 4ms/step - loss: 0.6581 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00948: val_acc did not improve from 0.61600
Epoch 949/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6582 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00949: val_acc did not improve from 0.61600
Epoch 950/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6582 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00950: val_acc did not improve from 0.61600
Epoch 951/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6578 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00951: val_acc did not improve from 0.61600
Epoch 952/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6576 - acc: 0.6320 - val_loss: 0.6666 - val_acc: 0.6160

Epoch 00952: val_acc did not improve from 0.61600
Epoch 953/1000
1000/1000 [==============================] - 4s 4ms/ste

1000/1000 [==============================] - 4s 4ms/step - loss: 0.6585 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00991: val_acc did not improve from 0.61600
Epoch 992/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6588 - acc: 0.6320 - val_loss: 0.6664 - val_acc: 0.6160

Epoch 00992: val_acc did not improve from 0.61600
Epoch 993/1000
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6583 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00993: val_acc did not improve from 0.61600
Epoch 994/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6583 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00994: val_acc did not improve from 0.61600
Epoch 995/1000
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6578 - acc: 0.6320 - val_loss: 0.6665 - val_acc: 0.6160

Epoch 00995: val_acc did not improve from 0.61600
Epoch 996/1000
1000/1000 [==============================] - 3s 3ms/ste

In [ ]:
model.load_weights("best-model_V7_2.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.evaluate(tt1,test_y,batch_size=32)